In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional

In [2]:
class MDP:
    """
    Markov Decision Process implementation with Value Iteration solver.
    """

    def __init__(self, states: List[int], actions: List[int],
                 transitions: Dict, rewards: Dict, gamma: float = 0.9):

        self.states = states
        self.actions = actions
        self.transitions = transitions
        self.rewards = rewards
        self.gamma = gamma
        self.n_states = len(states)
        self.n_actions = len(actions)

    def get_transition_prob(self, s: int, a: int, s_next: int) -> float:
        """Get transition probability P(s'|s,a)"""
        return self.transitions.get((s, a, s_next), 0.0)

    def get_reward(self, s: int, a: int, s_next: int) -> float:
        """Get reward R(s,a,s')"""
        return self.rewards.get((s, a, s_next), 0.0)

In [4]:
def value_iteration(self, theta: float = 1e-6, max_iterations: int = 1000) -> Tuple[np.ndarray, np.ndarray, List[float]]:
      
        V = np.zeros(self.n_states)
        deltas = []

        for iteration in range(max_iterations):
            delta = 0
            V_new = np.zeros(self.n_states)

            # Update value for each state
            for s in self.states:
                # Compute Q-values for all actions
                q_values = []
                for a in self.actions:
                    q_value = 0
                    for s_next in self.states:
                        prob = self.get_transition_prob(s, a, s_next)
                        reward = self.get_reward(s, a, s_next)
                        q_value += prob * (reward + self.gamma * V[s_next])
                    q_values.append(q_value)

                # Bellman optimality update
                V_new[s] = max(q_values)
                delta = max(delta, abs(V_new[s] - V[s]))

            V = V_new
            deltas.append(delta)

            # Check convergence
            if delta < theta:
                print(f"Value iteration converged after {iteration + 1} iterations")
                break

        # Extract optimal policy
        policy = self.extract_policy(V)

        return V, policy, deltas

In [5]:
def extract_policy(self, V: np.ndarray) -> np.ndarray:
        """
        Extract policy from value function.

        Args:
            V: Value function

        Returns:
            policy: Optimal action for each state
        """
        policy = np.zeros(self.n_states, dtype=int)

        for s in self.states:
            q_values = []
            for a in self.actions:
                q_value = 0
                for s_next in self.states:
                    prob = self.get_transition_prob(s, a, s_next)
                    reward = self.get_reward(s, a, s_next)
                    q_value += prob * (reward + self.gamma * V[s_next])
                q_values.append(q_value)

            policy[s] = np.argmax(q_values)

        return policy

In [6]:
def simulate_episode(self, policy: np.ndarray, start_state: int,
                        max_steps: int = 100) -> Tuple[List[int], List[int], List[float]]:
    
        states = [start_state]
        actions = []
        rewards = []

        current_state = start_state

        for _ in range(max_steps):
            # Select action according to policy
            action = policy[current_state]
            actions.append(action)

            # Sample next state based on transition probabilities
            next_state_probs = []
            next_states = []
            for s_next in self.states:
                prob = self.get_transition_prob(current_state, action, s_next)
                if prob > 0:
                    next_state_probs.append(prob)
                    next_states.append(s_next)

            if not next_states:
                break

            # Normalize probabilities
            next_state_probs = np.array(next_state_probs)
            next_state_probs /= next_state_probs.sum()

            # Sample next state
            next_state = np.random.choice(next_states, p=next_state_probs)

            # Get reward
            reward = self.get_reward(current_state, action, next_state)
            rewards.append(reward)

            states.append(next_state)
            current_state = next_state

        return states, actions, rewards

In [7]:
def create_gridworld_mdp(grid_size: int = 5, gamma: float = 0.9) -> MDP:

    states = list(range(grid_size * grid_size))
    actions = [0, 1, 2, 3]  # up, right, down, left

    def state_to_pos(s: int) -> Tuple[int, int]:
        return (s // grid_size, s % grid_size)

    def pos_to_state(row: int, col: int) -> int:
        return row * grid_size + col

    # Define goal and obstacles
    goal_state = pos_to_state(grid_size - 1, grid_size - 1)
    obstacles = [pos_to_state(2, 2), pos_to_state(3, 3)]

    transitions = {}
    rewards = {}

    # Define transitions and rewards
    for s in states:
        row, col = state_to_pos(s)

        for a in actions:
            # Determine next position based on action
            next_positions = []

            if a == 0:  # up
                next_row, next_col = max(0, row - 1), col
            elif a == 1:  # right
                next_row, next_col = row, min(grid_size - 1, col + 1)
            elif a == 2:  # down
                next_row, next_col = min(grid_size - 1, row + 1), col
            else:  # left
                next_row, next_col = row, max(0, col - 1)

            s_next = pos_to_state(next_row, next_col)

            # Stochastic transitions: 80% intended, 10% each perpendicular
            # Intended direction
            transitions[(s, a, s_next)] = 0.8

            # Perpendicular directions (simplified)
            transitions[(s, a, s)] = 0.2  # Stay in place

            # Set rewards
            if s_next == goal_state:
                rewards[(s, a, s_next)] = 10.0
            elif s_next in obstacles:
                rewards[(s, a, s_next)] = -5.0
            else:
                rewards[(s, a, s_next)] = -0.1

    return MDP(states, actions, transitions, rewards, gamma)

In [8]:
def visualize_results(mdp: MDP, V: np.ndarray, policy: np.ndarray,
                     deltas: List[float], grid_size: int = 5):
    """Visualize value function, policy, and convergence."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    # Plot value function
    V_grid = V.reshape(grid_size, grid_size)
    im1 = axes[0].imshow(V_grid, cmap='viridis')
    axes[0].set_title('Value Function')
    axes[0].set_xlabel('Column')
    axes[0].set_ylabel('Row')
    plt.colorbar(im1, ax=axes[0])

    # Add value text
    for i in range(grid_size):
        for j in range(grid_size):
            axes[0].text(j, i, f'{V_grid[i, j]:.1f}',
                        ha='center', va='center', color='white', fontsize=8)

    # Plot policy
    policy_grid = policy.reshape(grid_size, grid_size)
    arrows = {0: '↑', 1: '→', 2: '↓', 3: '←'}

    axes[1].imshow(np.zeros((grid_size, grid_size)), cmap='gray', vmin=0, vmax=1)
    axes[1].set_title('Optimal Policy')
    axes[1].set_xlabel('Column')
    axes[1].set_ylabel('Row')

    for i in range(grid_size):
        for j in range(grid_size):
            action = policy_grid[i, j]
            axes[1].text(j, i, arrows[action],
                        ha='center', va='center', fontsize=20)

    # Plot convergence
    axes[2].plot(deltas)
    axes[2].set_title('Convergence')
    axes[2].set_xlabel('Iteration')
    axes[2].set_ylabel('Max Value Change (δ)')
    axes[2].set_yscale('log')
    axes[2].grid(True)

    plt.tight_layout()
    plt.show()